## S/N budget estimator and exposure time calculator, adapted from Turyshev2025

### Utilities

In [1]:
import numpy as np

In [2]:
# Fundamental physical constants

h = 6.62607015e-34    # [J s ≡ J Hz^-1] Planck's constant
c = 299792458.        # [m/s] Light speed in vacuum


# Useful conversion factors

arcsec_per_radian = 60*60*180/np.pi
print(f"arcsec/radian = {arcsec_per_radian:.6e}")

arcsec/radian = 2.062648e+05


### Functional definition

In [3]:
def F_0_V(F_lambda_0_W_m_2_nm_1=3.631e-11, lam_nm=550.):

    """Adapted from Eq. (7) of Turyshev2025.
       V-band spectral photon flux density zero point for calibration from: https://www.astronomy.ohio-state.edu/martini.10/usefuldata.html."""

    lam_m = lam_nm * 1.e-9
    F_0_V_photons_m_2_s_1_nm_1 = F_lambda_0_W_m_2_nm_1 * lam_m / (h * c)

    return F_0_V_photons_m_2_s_1_nm_1

print(f"V-band spectral photon flux density zero point for calibration: F_0_V = {F_0_V():.6e} photons m^-2 s^-1 nm^-1")

V-band spectral photon flux density zero point for calibration: F_0_V = 1.005338e+08 photons m^-2 s^-1 nm^-1


In [4]:
def apparent_magnitude_to_flux(F_0point, mag):

    return F_0point * 10**(-0.4 * mag)

#### Signal estimation

In [5]:
def F_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld=27.77):

    """Adapted from Eq. (8) of Turyshev2025."""

    F_exoworld_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld)

    return F_exoworld_photons_m_2_s_1_nm_1

print(f"Spectral flux density from an exoworld with a V-band apparent magnitude = 27.77: F_exoworld = {F_exoworld(F_0_V()):.6e} photons m^-2 s^-1 nm^-1")

Spectral flux density from an exoworld with a V-band apparent magnitude = 27.77: F_exoworld = 7.839930e-04 photons m^-2 s^-1 nm^-1


In [6]:
def photon_flux_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld=27.77, A_m2=100., dlam_nm=50.):

    """Adapted from Eq. (9) of Turyshev2025."""

    photon_flux_exoworld_photons_m_2_s_1_nm_1 = F_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld) * A_m2 * dlam_nm

    return photon_flux_exoworld_photons_m_2_s_1_nm_1

print(f"Photon flux from an exoworld with a V-band apparent magnitude = 27.77, a light-gathering area = 100 m^2, in a spectral bandwidth = 50 nm: F_exoworld = {photon_flux_exoworld(F_0_V())} photons s^-1")

#####

photon_flux_exoworld_photons_m_2_s_1_nm_1_pixel_1 = photon_flux_exoworld(F_0_V()) / 100

print(f"Photon flux per pixel of a 10x10-pixel surface map from an exoworld with a V-band apparent magnitude = 27.77, a light-gathering area = 100 m^2, in a spectral bandwidth = 50 nm: F_exoworld = {photon_flux_exoworld_photons_m_2_s_1_nm_1_pixel_1:.6e} photons s^-1 pixel^-1")

Photon flux from an exoworld with a V-band apparent magnitude = 27.77, a light-gathering area = 100 m^2, in a spectral bandwidth = 50 nm: F_exoworld = 3.919965189626896 photons s^-1
Photon flux per pixel of a 10x10-pixel surface map from an exoworld with a V-band apparent magnitude = 27.77, a light-gathering area = 100 m^2, in a spectral bandwidth = 50 nm: F_exoworld = 3.919965e-02 photons s^-1 pixel^-1


#### Astrophysical backgrounds

In [7]:
def F_zodi(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2=23.):

    """Adapted from Turyshev2025."""

    F_zodi_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2)

    return F_zodi_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from the local zodiacal light (from interplanetary dust within the Solar System) with a V-band apparent magnitude = 23 arcsec^-2: F_zodi = {F_zodi(F_0_V()):.6e} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from the local zodiacal light (from interplanetary dust within the Solar System) with a V-band apparent magnitude = 23 arcsec^-2: F_zodi = 6.343256e-02 photons m^-2 s^-1 nm^-1 arcsec^-2


In [8]:
def F_exozodi(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2=23.):

    """Adapted from Turyshev2025.
       Assuming 8 times the Solar System's zodiacal cloud, from the 5--8 upper limit factor for a disk coplanar with the α Cen AB orbit, as reported by Beichman+2025 and Sanghi+2025."""

    F_exozodi_photons_m_2_s_1_nm_1_arcsec_2 = 8. * apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2)

    return F_exozodi_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from an exozodiacal (circumstellar) dust belt with a V-band apparent magnitude = 23 arcsec^-2: F_exozodi = {F_exozodi(F_0_V())} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from an exozodiacal (circumstellar) dust belt with a V-band apparent magnitude = 23 arcsec^-2: F_exozodi = 0.5074604415758602 photons m^-2 s^-1 nm^-1 arcsec^-2


In [9]:
def F_diffuse_Milky_Way(F_0_V_photons_m_2_s_1_nm_1, mV_MilkyWay_arcsec_2=24.):

    """Adapted from Turyshev2025."""

    F_MilkyWay_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_MilkyWay_arcsec_2)

    return F_MilkyWay_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from scattered interstellar dust within the Milky Way with a V-band apparent magnitude = 24 arcsec^-2: F_diffuse_Milky_Way = {F_diffuse_Milky_Way(F_0_V()):.6e} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from scattered interstellar dust within the Milky Way with a V-band apparent magnitude = 24 arcsec^-2: F_diffuse_Milky_Way = 2.525296e-02 photons m^-2 s^-1 nm^-1 arcsec^-2


In [10]:
def F_extragalactic_background(F_0_V_photons_m_2_s_1_nm_1, mV_extragalactic_arcsec_2=26.5):

    """Adapted from Turyshev2025."""

    F_extragalactic_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_extragalactic_arcsec_2)

    return F_extragalactic_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from the integrated extragalactic isotropic background emission (from unresolved galaxies) with a V-band apparent magnitude = 26.5 arcsec^-2: F_extragalactic_background = {F_extragalactic_background(F_0_V()):.6e} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from the integrated extragalactic isotropic background emission (from unresolved galaxies) with a V-band apparent magnitude = 26.5 arcsec^-2: F_extragalactic_background = 2.525296e-03 photons m^-2 s^-1 nm^-1 arcsec^-2


In [11]:
def F_unresolved_MW_stars(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_stars=27.5):

    """Adapted from Turyshev2025."""

    F_unresolved_stars_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_stars)

    return F_unresolved_stars_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from unresolved Milky-Way stars with a V-band apparent magnitude = 27.5 arcsec^-2: F_unresolved_MW_stars = {F_unresolved_MW_stars(F_0_V()):.6e} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from unresolved Milky-Way stars with a V-band apparent magnitude = 27.5 arcsec^-2: F_unresolved_MW_stars = 1.005338e-03 photons m^-2 s^-1 nm^-1 arcsec^-2


In [12]:
def F_unresolved_extragalactic_point_sources(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_extragalactic_point_sources=28.):

    """Adapted from Turyshev2025."""

    F_unresolved_extragalactic_point_sources_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_extragalactic_point_sources)

    return F_unresolved_extragalactic_point_sources_photons_m_2_s_1_nm_1_arcsec_2

print(f"Spectral flux density from unresolved extragalactic point sources (mainly galaxies or AGNs) with a V-band apparent magnitude = 28 arcsec^-2: F_unresolved_extragalactic_point_sources = {F_unresolved_extragalactic_point_sources(F_0_V()):.6e} photons m^-2 s^-1 nm^-1 arcsec^-2")

Spectral flux density from unresolved extragalactic point sources (mainly galaxies or AGNs) with a V-band apparent magnitude = 28 arcsec^-2: F_unresolved_extragalactic_point_sources = 6.343256e-04 photons m^-2 s^-1 nm^-1 arcsec^-2


In [13]:
F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2 = np.sum([F_zodi(F_0_V()), F_exozodi(F_0_V()),
                                                                    F_diffuse_Milky_Way(F_0_V()), F_unresolved_MW_stars(F_0_V()),
                                                                    F_extragalactic_background(F_0_V()), F_unresolved_extragalactic_point_sources(F_0_V())])

print(f"Total spectral flux density from all astrophysical background sources: F_astrophysical_background = {F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2} photons m^-2 s^-1 nm^-1 arcsec^-2")

Total spectral flux density from all astrophysical background sources: F_astrophysical_background = 0.6003109111514279 photons m^-2 s^-1 nm^-1 arcsec^-2


In [14]:
def Airy_core_solid_angle_arcsec2(D_m=2*np.sqrt(100./np.pi), lam_nm=550.):

    """Adapted from Turyshev2025."""

    lam_m = lam_nm * 1e-9
    theta_rad = 1.22 * lam_m / D_m
    theta_arcsec = theta_rad * arcsec_per_radian
    Omega_core_arcsec2 = np.pi * (theta_arcsec**2)

    return Omega_core_arcsec2

print(f"Diffraction-limited Airy PSF's core solid angle: {Airy_core_solid_angle_arcsec2():.6e} arcsec^2")

Diffraction-limited Airy PSF's core solid angle: 4.726450e-04 arcsec^2


In [15]:
def astrophysical_background_photon_flux(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2, A_m2=100., D_m=2*np.sqrt(100./np.pi), dlam_nm=50., lam_nm=550.):

    """Adapted from Eq. (12) of Turyshev2025."""

    astrophysical_background_photon_flux_photons_s_1 = F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2 * A_m2 * dlam_nm * Airy_core_solid_angle_arcsec2(D_m, lam_nm)

    return astrophysical_background_photon_flux_photons_s_1

print(f"Total astrophysical-background photon flux: {astrophysical_background_photon_flux(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2):.6e} photons s^-1")

Total astrophysical-background photon flux: 1.418670e+00 photons s^-1


#### Detector noise

In [16]:
dark_current_electrons_s_1_pixel = 1.e-4
N_pixels_photometric_aperture = 4
dark_current_electrons_s_1 = N_pixels_photometric_aperture * dark_current_electrons_s_1_pixel

print(f"Dark current over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels: {dark_current_electrons_s_1} electrons s^-1")

#####

CIC_rate_electrons_frame_1_pixel_1 = 5.e-3
frame_rate_s_1 = 1
CIC_rate_electrons_s_1 = N_pixels_photometric_aperture * CIC_rate_electrons_frame_1_pixel_1 * frame_rate_s_1

print(f"Clock-induced charge rate over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels, given a frame rate of {int(frame_rate_s_1)} Hz: {CIC_rate_electrons_s_1} electrons s^-1")

#####

sigma_readout_electrons_frame_1_pixel_1 = .1
readout_noise = N_pixels_photometric_aperture * (sigma_readout_electrons_frame_1_pixel_1) * frame_rate_s_1    # To check

print(f"Read-out noise over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels, given a frame rate of {int(frame_rate_s_1)} Hz: {readout_noise} electrons s^-1")

#####

detector_noise_electrons_s_1 = dark_current_electrons_s_1 + CIC_rate_electrons_s_1 + readout_noise

print(f"\nTotal detector noise over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels, given a frame rate of {int(frame_rate_s_1)} Hz: {detector_noise_electrons_s_1} electrons s^-1")

Dark current over a photometric aperture of 4 pixels: 0.0004 electrons s^-1
Clock-induced charge rate over a photometric aperture of 4 pixels, given a frame rate of 1 Hz: 0.02 electrons s^-1
Read-out noise over a photometric aperture of 4 pixels, given a frame rate of 1 Hz: 0.4 electrons s^-1

Total detector noise over a photometric aperture of 4 pixels, given a frame rate of 1 Hz: 0.4204 electrons s^-1


#### Stellar leakage

In [17]:
def starlight_leakage_photon_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star=4.83, A_m2=100., dlam_nm=50., residual_raw_C=1e-8):

    """Adapted from Eq. (13) of Turyshev2025."""

    F_star_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star)

    starlight_leakage_photon_flux_photons_s_1 = F_star_photons_m_2_s_1_nm_1 * A_m2 * dlam_nm * residual_raw_C

    return starlight_leakage_photon_flux_photons_s_1

print(f"Starlight leakage photon flux into the exoworld Airy PSF's core, within the telescope: {starlight_leakage_photon_flux(F_0_V_photons_m_2_s_1_nm_1=F_0_V())} photons s^-1")

Starlight leakage photon flux into the exoworld Airy PSF's core, within the telescope: 58.78712350580647 photons s^-1


In [18]:
def quasistatic_speckle_drift_photon_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star=4.83, A_m2=100., dlam_nm=50.):

    """Adapted from Eq. (14) of Turyshev2025."""

    F_star_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star)

    raw_C_degradation = 1.e-9

    quasistatic_speckle_drift_photon_flux_photons_s_1 = np.sqrt(F_star_photons_m_2_s_1_nm_1 * A_m2 * dlam_nm * raw_C_degradation)

    return quasistatic_speckle_drift_photon_flux_photons_s_1

print(f"Quasi-static speckle drift photon flux: {quasistatic_speckle_drift_photon_flux(F_0_V_photons_m_2_s_1_nm_1=F_0_V())} photons s^-1")

Quasi-static speckle drift photon flux: 2.4246056072237083 photons s^-1


#### Other miscellanea: pointing jitter, quantization noise, cosmic rays, and thermal emission

In [19]:
pointing_jitter_count_rate_electrons_s_1 = .3 * (starlight_leakage_photon_flux(F_0_V()) + astrophysical_background_photon_flux(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2))

print(f"Spacecraft's pointing jitter count rate: {pointing_jitter_count_rate_electrons_s_1} electrons s^-1")

Spacecraft's pointing jitter count rate: 18.061737970584034 electrons s^-1


In [20]:
sigma_quantization_RMSelectrons_pixel_1 = .5
quantization_noise_electrons_s_1 = N_pixels_photometric_aperture * (sigma_quantization_RMSelectrons_pixel_1**2) * frame_rate_s_1

print(f"Quantization noise count rate over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels, given a frame rate of {int(frame_rate_s_1)} Hz: {quantization_noise_electrons_s_1} electrons s^-1")

Quantization noise count rate over a photometric aperture of 4 pixels, given a frame rate of 1 Hz: 1.0 electrons s^-1


In [21]:
deep_space_cosmic_ray_count_rate_cm_2_s_1 = 5
pixel_pitch_cm = 1.e-4
cosmic_ray_count_rate_electrons_s_1 = deep_space_cosmic_ray_count_rate_cm_2_s_1 * (.5 * N_pixels_photometric_aperture * pixel_pitch_cm)**2

print(f"Representative count rate from deep-space cosmic-ray-generated events over a photometric aperture of {int(N_pixels_photometric_aperture)} pixels: {cosmic_ray_count_rate_electrons_s_1} electrons s^-1")

Representative count rate from deep-space cosmic-ray-generated events over a photometric aperture of 4 pixels: 2e-07 electrons s^-1


In [22]:
# def thermal_emission_primary_mirror    # Above ~800 nm

### S/N and ETC

#### Validation

In [23]:
throughput_optics_instrumentation = .2
duty_cycle = .5
throughput_system = throughput_optics_instrumentation * duty_cycle

print(f"End-to-end (parametrized) system throughput: {throughput_system}")

End-to-end (parametrized) system throughput: 0.1


In [24]:
signal_count_rate = throughput_system * photon_flux_exoworld(F_0_V())

print(f"Signal count rate from an exoworld with a V-band apparent magnitude = 27.77: {signal_count_rate} electrons s^-1, assuming a 1-to-1 photo-electron conversion")

signal_count_rate_surface_map_pixel_1 = signal_count_rate / 100

print(f"Signal count rate per pixel from an exoworld with a V-band apparent magnitude = 27.77 for a 10x10-pixel surface map: {signal_count_rate_surface_map_pixel_1} electrons s^-1 pixel^-1, assuming a 1-to-1 photo-electron conversion")

Signal count rate from an exoworld with a V-band apparent magnitude = 27.77: 0.3919965189626896 electrons s^-1, assuming a 1-to-1 photo-electron conversion
Signal count rate per pixel from an exoworld with a V-band apparent magnitude = 27.77 for a 10x10-pixel surface map: 0.003919965189626896 electrons s^-1 pixel^-1, assuming a 1-to-1 photo-electron conversion


In [25]:
instrumentation_count_rate = quasistatic_speckle_drift_photon_flux(F_0_V()) + pointing_jitter_count_rate_electrons_s_1 + quantization_noise_electrons_s_1 + cosmic_ray_count_rate_electrons_s_1

noise_count_rate = detector_noise_electrons_s_1 + instrumentation_count_rate + throughput_system * (starlight_leakage_photon_flux(F_0_V()) + astrophysical_background_photon_flux(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2))

print(f"Total noise count rate, assuming a 1-to-1 conversion for photo-generated electrons: {noise_count_rate} electrons s^-1")


Total noise count rate, assuming a 1-to-1 conversion for photo-generated electrons: 27.92732310133575 electrons s^-1


In [26]:
SNR = signal_count_rate / np.sqrt(signal_count_rate + noise_count_rate)

print(f"SNR: {SNR}")

#####

SNR_surface_map_pixel_1 = signal_count_rate_surface_map_pixel_1 / np.sqrt(signal_count_rate_surface_map_pixel_1 + noise_count_rate)

print(f"SNR per pixel of a 10x10-pixel surface map: {SNR_surface_map_pixel_1} pixel^-1")

SNR: 0.07366154147109955
SNR per pixel of a 10x10-pixel surface map: 0.0007417150277895989 pixel^-1


In [27]:
SNR_detection_threshold = 5.

#####

t_integration_s = (SNR_detection_threshold / SNR)**2

print(f"Integration time required to achieve an SNR of {int(SNR_detection_threshold)}: {t_integration_s/(60*60):.3f} hours ({t_integration_s/(60*60*24):.3f} days)")

#####

t_integration_surface_map_s_pixel_1 = (SNR_detection_threshold / SNR_surface_map_pixel_1)**2

print(f"Integration time required to achieve an SNR of {int(SNR_detection_threshold)} per pixel of a 10x10-pixel surface map: {t_integration_surface_map_s_pixel_1/(60*60):.3f} hours ({t_integration_surface_map_s_pixel_1/(60*60*24):.3f} days)")

Integration time required to achieve an SNR of 5: 1.280 hours (0.053 days)
Integration time required to achieve an SNR of 5 per pixel of a 10x10-pixel surface map: 12623.022 hours (525.959 days)


#### Case studies

In [28]:
def budgeting(
                F_lambda_0_W_m_2_nm_1,
                SNR_detection_threshold,
                A_m2,
                D_m,
                lam_nm,
                dlam_nm,
                throughput_system,
                residual_raw_C,
                pixel_pitch_cm,
                N_pixels_photometric_aperture,
                dark_current_electrons_s_1_pixel,
                CIC_rate_electrons_frame_1_pixel_1,
                frame_rate_s_1,
                sigma_readout_electrons_frame_1_pixel_1,
                sigma_quantization_RMSelectrons_pixel_1,
                pointing_jitter_fraction,
                mV_exoworld,
                mV_star,
                mV_zodi_arcsec_2,
                mV_MilkyWay_arcsec_2,
                mV_extragalactic_arcsec_2,
                mV_unresolved_stars,
                mV_unresolved_extragalactic_point_sources,
                deep_space_cosmic_ray_count_rate_cm_2_s_1,
    ):

    #####

    F0V = F_0_V(F_lambda_0_W_m_2_nm_1=F_lambda_0_W_m_2_nm_1, lam_nm=lam_nm)

    #####

    signal_count_rate = throughput_system * photon_flux_exoworld(F0V, mV_exoworld=mV_exoworld, A_m2=A_m2, dlam_nm=dlam_nm)

    #####

    zodi_flux = F_zodi(F0V, mV_zodi_arcsec_2)
    exozodi_flux = F_exozodi(F0V, mV_zodi_arcsec_2)
    diffuse_MW_flux = F_diffuse_Milky_Way(F0V, mV_MilkyWay_arcsec_2)
    extragalactic_background_flux = F_extragalactic_background(F0V, mV_extragalactic_arcsec_2)
    unresolved_MWstars_flux = F_unresolved_MW_stars(F0V, mV_unresolved_stars)
    unresolved_extragalactic_point_sources_flux = F_unresolved_extragalactic_point_sources(F0V, mV_unresolved_extragalactic_point_sources)

    astrophysical_background_flux = np.sum([zodi_flux, exozodi_flux, diffuse_MW_flux, extragalactic_background_flux, unresolved_MWstars_flux, unresolved_extragalactic_point_sources_flux])

    astro_background_photon_flux = astrophysical_background_photon_flux(
        astrophysical_background_flux,
        A_m2=A_m2,
        D_m=D_m,
        dlam_nm=dlam_nm,
        lam_nm=lam_nm,
    )

    #####

    stellar_leakage_photon_flux = starlight_leakage_photon_flux(
        F0V,
        mV_star=mV_star,
        A_m2=A_m2,
        dlam_nm=dlam_nm,
        residual_raw_C=residual_raw_C,
    )

    speckle_drift_photon_flux = quasistatic_speckle_drift_photon_flux(
        F0V,
        mV_star=mV_star,
        A_m2=A_m2,
        dlam_nm=dlam_nm,
    )

    pointing_jitter_count_rate_electrons_s_1 = pointing_jitter_fraction * (stellar_leakage_photon_flux + astro_background_photon_flux)

    quantization_noise_electrons_s_1 = N_pixels_photometric_aperture * (sigma_quantization_RMSelectrons_pixel_1**2) * frame_rate_s_1

    cosmic_ray_count_rate_electrons_s_1 = deep_space_cosmic_ray_count_rate_cm_2_s_1 * (.5 * N_pixels_photometric_aperture * pixel_pitch_cm)**2

    instrumentation_count_rate = speckle_drift_photon_flux + pointing_jitter_count_rate_electrons_s_1 + quantization_noise_electrons_s_1 + cosmic_ray_count_rate_electrons_s_1

    #####

    dark_current_electrons_s_1 = N_pixels_photometric_aperture * dark_current_electrons_s_1_pixel
    CIC_rate_electrons_s_1 = N_pixels_photometric_aperture * CIC_rate_electrons_frame_1_pixel_1 * frame_rate_s_1
    readout_noise = N_pixels_photometric_aperture * (sigma_readout_electrons_frame_1_pixel_1) * frame_rate_s_1    # To check
    detector_noise_electrons_s_1 = dark_current_electrons_s_1 + CIC_rate_electrons_s_1 + readout_noise

    noise_count_rate = detector_noise_electrons_s_1 + instrumentation_count_rate + throughput_system * (stellar_leakage_photon_flux + astro_background_photon_flux)

    #####

    SNR = signal_count_rate / np.sqrt(signal_count_rate + noise_count_rate)

    #####

    t_integration_s = (SNR_detection_threshold / SNR)**2

    #####

    return {
        "F_0_V": F0V,
        "signal_count_rate": signal_count_rate,
        "noise_count_rate": noise_count_rate,
        "SNR": SNR,
        "t_integration": t_integration_s,
    }


In [29]:
# Analysis parameters

F_lambda_0_W_m_2_nm_1 = 3.631e-11                              # [W m^-2 nm^-1] ≡ [erg s^-1 cm^-2 Å^-1] V-band spectral flux density zero point for calibration from: https://www.astronomy.ohio-state.edu/martini.10/usefuldata.html
SNR_detection_threshold = 5.


# Telescopic parameters

#D_m = 50.                                                      # Telescope diameter
#A_m2 = np.pi * (.5*D_m)**2                                     # Light-gathering area
A_m2 = 100.                                                    # Light-gathering area
D_m = 2. * np.sqrt(A_m2 / np.pi)                               # Diameter
lam_nm = 550.                                                  # Reference wavelength
dlam_nm = 50.                                                  # Bandwidth

throughput_optics_instrumentation = .20                        # Instrument throughput [~0.15--0.25]
duty_cycle = .5                                                # on-target fraction; pessimistic ~0.3, optimistic ~0.6
throughput_system = throughput_optics_instrumentation * duty_cycle    # End-to-end system throughput

residual_raw_C = 1.e-8                                         # Residual raw contrast [1e-9--1e-8]
#raw_C_degradation = 1.e-9                                      # Raw contrast degradation [1e-10--1e-8]

pixel_pitch_cm = 1.e-4                                         # Pixel pitch
N_pixels_photometric_aperture = 4                              # Photometric aperture's sampling pixels
dark_current_electrons_s_1_pixel = 1.e-4                       # Dark current [~1e-4--1e-3]
CIC_rate_electrons_frame_1_pixel_1 = 5.e-3                     # Electron-multiplying charge-coupled device (EMCCD) clock-induced charge rate [~1e-3--5e-3]
frame_rate_s_1 = 1.                                            # Frame rate [1--10 Hz]
sigma_readout_electrons_frame_1_pixel_1 = .1                   # Read-out electronic noise [~0.01--0.1]
sigma_quantization_RMSelectrons_pixel_1 = .5                   # Quantization error [~0.1--0.5]

pointing_jitter_fraction = .3                                  # Pointing jitter fraction of the starlight leakage and astrophysical background photon fluxes [~0.1--0.3]


# Astrophysical parameters

mV_exoworld = 27.77                                            # Target exoworld's apparent magnitude
mV_star = 4.83                                                 # Host star's apparent magnitude

mV_zodi_arcsec_2 = 23.                                         # Solar system's zodiacal light's apparent magnitude [~22.5--23]
#exozodi_factor = 8.                                            # Exozodiacal light factor [~3--10]
mV_MilkyWay_arcsec_2 = 24.                                     # Diffuse Milky Way's apparent magnitude [~23--25]
mV_extragalactic_arcsec_2 = 26.5                               # Extragalactic background light's apparent magnitude [~25.5--27]
mV_unresolved_stars = 27.5                                     # Unresolved Milky Way stars' apparent magnitude
mV_unresolved_extragalactic_point_sources = 28.                # Unresolved extragalactic point sources' apparent magnitude

deep_space_cosmic_ray_count_rate_cm_2_s_1 = 5.                 # Deep-space cosmic-ray-generated count rate

In [30]:
if __name__ == "__main__":

    results = budgeting(
                        F_lambda_0_W_m_2_nm_1,
                        SNR_detection_threshold,
                        A_m2,
                        D_m,
                        lam_nm,
                        dlam_nm,
                        throughput_system,
                        residual_raw_C,
                        pixel_pitch_cm,
                        N_pixels_photometric_aperture,
                        dark_current_electrons_s_1_pixel,
                        CIC_rate_electrons_frame_1_pixel_1,
                        frame_rate_s_1,
                        sigma_readout_electrons_frame_1_pixel_1,
                        sigma_quantization_RMSelectrons_pixel_1,
                        pointing_jitter_fraction,
                        mV_exoworld,
                        mV_star,
                        mV_zodi_arcsec_2,
                        mV_MilkyWay_arcsec_2,
                        mV_extragalactic_arcsec_2,
                        mV_unresolved_stars,
                        mV_unresolved_extragalactic_point_sources,
                        deep_space_cosmic_ray_count_rate_cm_2_s_1,
    )

    print("\n========= S/N budget and exposure time =========\n")
    print(f"F_0_V                                           : {results['F_0_V']:.6e} photons m^-2 s^-1 nm^-1")
    print(f"signal_count_rate                               : {results['signal_count_rate']} photo-electrons s^-1")
    print(f"noise_count_rate                                : {results['noise_count_rate']} photo-electrons s^-1")
    print(f"SNR (detection threshold SNR = {SNR_detection_threshold})             : {results['SNR']}")
    print(f"t_integration                                   : {results['t_integration'] / (60*60):.3f} hours ({results['t_integration'] / (60*60*24):.3f} days)\n")
    print("================================================\n")


========= S/N budget and exposure time =========

F_0_V                                           : 1.005338e+08 photons m^-2 s^-1 nm^-1
signal_count_rate                               : 0.3919965189626896 photo-electrons s^-1
noise_count_rate                                : 27.92732310133575 photo-electrons s^-1
SNR (detection threshold SNR = 5.0)             : 0.07366154147109955
t_integration                                   : 1.280 hours (0.053 days)




In [31]:
# Telescopic parameters

#D_m = 3.                                                      # Telescope diameter
#A_m2 = np.pi * (.5*D_m)**2                                     # Light-gathering area

In [32]:
# Telescopic parameters

#D_m = 10.                                                      # Telescope diameter
#A_m2 = np.pi * (.5*D_m)**2                                     # Light-gathering area

In [33]:
# Telescopic parameters

#D_m = 25.                                                      # Telescope diameter
#A_m2 = np.pi * (.5*D_m)**2                                     # Light-gathering area

In [34]:
# Telescopic parameters

#D_m = 50.                                                      # Telescope diameter
#A_m2 = np.pi * (.5*D_m)**2                                     # Light-gathering area